<a href="https://colab.research.google.com/github/sruthi1996/DeepLearning/blob/master/p1ch4_tabularwine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import torch
import imageio
import csv
torch.set_printoptions(edgeitems=2, threshold=50)

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
download = drive.CreateFile({'id':"1hDnEYeFSlT2MtjPmxK5pal7AIuWmljzy"})
download.GetContentFile('wineQualityWhites.csv')

In [0]:
import pandas as pd
data = pd.read_csv('wineQualityWhites.csv')

In [74]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            4898 non-null   int64  
 1   fixed.acidity         4898 non-null   float64
 2   volatile.acidity      4898 non-null   float64
 3   citric.acid           4898 non-null   float64
 4   residual.sugar        4898 non-null   float64
 5   chlorides             4898 non-null   float64
 6   free.sulfur.dioxide   4898 non-null   float64
 7   total.sulfur.dioxide  4898 non-null   float64
 8   density               4898 non-null   float64
 9   pH                    4898 non-null   float64
 10  sulphates             4898 non-null   float64
 11  alcohol               4898 non-null   float64
 12  quality               4898 non-null   int64  
dtypes: float64(11), int64(2)
memory usage: 497.6 KB


In [75]:
num_array = data.values
num_array

array([[1.000e+00, 7.000e+00, 2.700e-01, ..., 4.500e-01, 8.800e+00,
        6.000e+00],
       [2.000e+00, 6.300e+00, 3.000e-01, ..., 4.900e-01, 9.500e+00,
        6.000e+00],
       [3.000e+00, 8.100e+00, 2.800e-01, ..., 4.400e-01, 1.010e+01,
        6.000e+00],
       ...,
       [4.896e+03, 6.500e+00, 2.400e-01, ..., 4.600e-01, 9.400e+00,
        6.000e+00],
       [4.897e+03, 5.500e+00, 2.900e-01, ..., 3.800e-01, 1.280e+01,
        7.000e+00],
       [4.898e+03, 6.000e+00, 2.100e-01, ..., 3.200e-01, 1.180e+01,
        6.000e+00]])

In [76]:
for col in data.columns: 
    print(col) 

Unnamed: 0
fixed.acidity
volatile.acidity
citric.acid
residual.sugar
chlorides
free.sulfur.dioxide
total.sulfur.dioxide
density
pH
sulphates
alcohol
quality


In [77]:
list(data.columns) 

['Unnamed: 0',
 'fixed.acidity',
 'volatile.acidity',
 'citric.acid',
 'residual.sugar',
 'chlorides',
 'free.sulfur.dioxide',
 'total.sulfur.dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol',
 'quality']

In [78]:
num_array.shape

(4898, 13)

In [0]:
wineq = torch.from_numpy(num_array)


In [80]:
wineq.shape, wineq.dtype

(torch.Size([4898, 13]), torch.float64)

In [81]:
data = wineq[:, :-2] 
data, data.shape

(tensor([[1.0000e+00, 7.0000e+00,  ..., 3.0000e+00, 4.5000e-01],
         [2.0000e+00, 6.3000e+00,  ..., 3.3000e+00, 4.9000e-01],
         ...,
         [4.8970e+03, 5.5000e+00,  ..., 3.3400e+00, 3.8000e-01],
         [4.8980e+03, 6.0000e+00,  ..., 3.2600e+00, 3.2000e-01]],
        dtype=torch.float64), torch.Size([4898, 11]))

In [82]:
target = wineq[:, -1] # <2>
target, target.shape

(tensor([6., 6.,  ..., 7., 6.], dtype=torch.float64), torch.Size([4898]))

In [83]:
target = wineq[:, -1].long()
target

tensor([6, 6,  ..., 7, 6])

In [84]:
#onehot encoding target label
target_onehot = torch.zeros(target.shape[0], 10)
target_onehot.scatter_(1, target.unsqueeze(1), 1.0)

tensor([[0., 0.,  ..., 0., 0.],
        [0., 0.,  ..., 0., 0.],
        ...,
        [0., 0.,  ..., 0., 0.],
        [0., 0.,  ..., 0., 0.]])

In [85]:
target_unsqueezed = target.unsqueeze(1)
target_unsqueezed

tensor([[6],
        [6],
        ...,
        [7],
        [6]])

In [86]:
#mean
data_mean = torch.mean(data, dim=0)
data_mean

tensor([2.4495e+03, 6.8548e+00, 2.7824e-01, 3.3419e-01, 6.3914e+00, 4.5772e-02,
        3.5308e+01, 1.3836e+02, 9.9403e-01, 3.1883e+00, 4.8985e-01],
       dtype=torch.float64)

In [87]:
#variance
data_var = torch.var(data, dim=0)
data_var

tensor([1.9996e+06, 7.1211e-01, 1.0160e-02, 1.4646e-02, 2.5726e+01, 4.7733e-04,
        2.8924e+02, 1.8061e+03, 8.9455e-06, 2.2801e-02, 1.3025e-02],
       dtype=torch.float64)

In [88]:
#normalize
data_normalized = (data - data_mean) / torch.sqrt(data_var)
data_normalized

tensor([[-1.7315e+00,  1.7208e-01,  ..., -1.2468e+00, -3.4915e-01],
        [-1.7308e+00, -6.5743e-01,  ...,  7.3995e-01,  1.3417e-03],
        ...,
        [ 1.7308e+00, -1.6054e+00,  ...,  1.0049e+00, -9.6251e-01],
        [ 1.7315e+00, -1.0129e+00,  ...,  4.7505e-01, -1.4882e+00]],
       dtype=torch.float64)

In [89]:
bad_indexes = target <= 3 
bad_indexes.shape, bad_indexes.dtype, bad_indexes.sum()

(torch.Size([4898]), torch.bool, tensor(20))

In [90]:
#bad data
bad_data = data[bad_indexes]
bad_data.shape

torch.Size([20, 11])

In [92]:
total_sulfur_threshold = 141.83
total_sulfur_data = data[:,6]
predicted_indexes = torch.lt(total_sulfur_data, total_sulfur_threshold)

predicted_indexes.shape, predicted_indexes.dtype, predicted_indexes.sum()

(torch.Size([4898]), torch.bool, tensor(4896))

In [93]:
actual_indexes = target > 5
actual_indexes.shape, actual_indexes.dtype, actual_indexes.sum()

(torch.Size([4898]), torch.bool, tensor(3258))

In [94]:
n_matches = torch.sum(actual_indexes & predicted_indexes).item()
n_predicted = torch.sum(predicted_indexes).item()
n_actual = torch.sum(actual_indexes).item()
n_matches, n_matches / n_predicted, n_matches / n_actual

(3258, 0.6654411764705882, 1.0)